#  Считаем NPV и IRR методом Монте-Карло

In [2]:
import numpy as np
import numpy.random as npr
import timeit

npr.seed(69)


### Подсчет обычным методом 

In [5]:
price = np.array([6, 6.05, 6.1])
volume = np.array([802000, 967000, 1132000])
cost_coef = 0.55
oper_coef = 0.15

cash_flow = ( 1 -  cost_coef ) * (1 - oper_coef ) * ( 1 - 0.32) * price * volume   
cash_flow = np.append(-3400000, cash_flow )
npv = np.npv(0.1,cash_flow)
irr = np.irr(cash_flow)

print(npv)
print(irr)

344796.1010518393
0.15331410149124514


### (а) С циклом

In [4]:
def npv_mc(y, numb_iter):
    
    npv_all = []
    irr_all = [] 
        
    
    for i in np.arange(numb_iter):    
        
        price = npr.triangular([5.9,5.95,6], [6,6.05,6.1],  [6.1,6.15,6.2] , 3)
        volume = npr.normal([802,967,1132], [25,30,25], 3) * 1000
        cost_coef = npr.triangular(0.5, 0.55, 0.65) 
        oper_coef = npr.normal(0.15, 0.02) 
        
        cash_flow = ( 1 - cost_coef ) * (1 - oper_coef ) * ( 1 - 0.32) * price * volume   
        cash_flow = np.append(-3400000, cash_flow )  
        
        npv_all = np.append(npv_all, np.npv(0.1,cash_flow))
        irr_all = np.append(irr_all, np.irr(cash_flow))
    
    if y == 1:
        return npv_all
    else:
        return irr_all

npv_all1 = npv_mc(1, 10000)
irr_all1 = npv_mc(2, 10000)
        
print('Среднее NPV:', np.mean(npv_all1))
print('Среднее IRR:', np.mean(irr_all1))

Среднее NPV: 211073.82845253867
Среднее IRR: 0.1316342623578176


### Графики

In [ ]:
import matplotlib.pyplot as plt 

plt.hist(npv_all1, bins= 30, align='mid', histtype='bar', facecolor='green') 
plt.title('Net present value') 
plt.axvline(x=np.mean(npv_all1),linestyle='dashed',color='blue') 
plt.axvline(x=np.percentile(npv_all1, 10),linestyle='dashed',color='orange') 
plt.axvline(x=np.percentile(npv_all1, 50),linestyle='dashed',color='orange') 
plt.axvline(x=np.percentile(npv_all1, 90),linestyle='dashed',color='orange') 
plt.show()


plt.boxplot(npv_all1, whis=[1, 99],  widths=0.2)
plt.title('Net present value') 
plt.show()


plt.hist(irr_all1, bins=30, align='mid',histtype='bar', facecolor='yellow') 
plt.title('Internal rate of return') 
plt.axvline(x=np.mean(irr_all1),linestyle='dashed',color='blue') 
plt.axvline(x=np.percentile(irr_all1, 10),linestyle='dashed',color='orange') 
plt.axvline(x=np.percentile(irr_all1, 50),linestyle='dashed',color='orange') 
plt.axvline(x=np.percentile(irr_all1, 90),linestyle='dashed',color='orange') 
plt.show()

plt.boxplot(irr_all1, whis=[1, 99],  widths=0.2)
plt.title('Internal rate of return') 
plt.show()

### (б) С вложенным циклом 

In [10]:
def npv_mc_no(y, numb_iter):
   
    npv_all = []
    irr_all = [] 
    
    price1 = np.random.triangular(5.9, 6, 6.1, numb_iter)
    price2 = np.random.triangular(5.95, 6.05, 6.15, numb_iter)
    price3 = np.random.triangular(6, 6.1, 6.2, numb_iter) 
    
    volume1 = np.random.normal(802, 25, numb_iter) * 1000
    volume2 = np.random.normal(967, 30, numb_iter) * 1000
    volume3 = np.random.normal(1132, 25, numb_iter) * 1000
        
    cost_coef = npr.triangular(0.5, 0.55, 0.65,numb_iter) 
    oper_coef = npr.normal(0.15, 0.02,numb_iter) 
    
    npv_all = np.array( [ np.npv(0.1, np.append(-3400000, ( 1 - 0.32)*(1 - oper_coef[i])*(1 - cost_coef[i])*np.array([price1[i]*volume1[i], price2[i]*volume2[i], price3[i]*volume3[i]])))  for i in np.arange(numb_iter) ])
    irr_all = np.array( [ np.irr(np.append(-3400000, ( 1 - 0.32)*(1 - oper_coef[i])*(1 - cost_coef[i])*np.array([price1[i]*volume1[i], price2[i]*volume2[i], price3[i]*volume3[i]])))  for i in np.arange(numb_iter) ] )  

    if y == 1:
        return npv_all
    else:
        return irr_all
    
    
    
npv_all2 = npv_mc_no(1, 10000)
irr_all2 = npv_mc_no(2, 10000)
        
print('Среднее NPV:', np.mean(npv_all2))
print('Среднее IRR:', np.mean(irr_all2))
        

Среднее NPV: 203202.45559004552
Среднее IRR: 0.1311681714067047


### (в) С циклом и с вложенным циклом

In [13]:
def npv_mc_no_no(y, numb_iter):
    
    npv_all = []
    irr_all = [] 
    profit = []
    
    for i in np.arange(numb_iter):
        profit.append(npr.triangular([5.9,5.95,6], [6,6.05,6.1],  [6.1,6.15,6.2] , 3)  * npr.normal([802,967,1132], [25,30,25], 3) * 1000)      
    
    cost_coef = npr.triangular(0.5, 0.55, 0.65,numb_iter) 
    oper_coef = npr.normal(0.15, 0.02,numb_iter) 
    
    npv_all = np.array([ np.npv(0.1, np.append(-3400000, ( 1 - 0.32)*(1 - oper_coef[i])*(1 - cost_coef[i]) * profit[i] ))  for i in np.arange(numb_iter)  ])
    irr_all = np.array([  np.irr(np.append(-3400000, ( 1 - 0.32)*(1 - oper_coef[i])*(1 - cost_coef[i])* profit[i] ))  for i in np.arange(numb_iter)])  
    
    if y == 1:
        return npv_all
    else:
        return irr_all

npv_all3 = npv_mc_no_no(1, 10000)
irr_all3 = npv_mc_no_no(2, 10000)   

print('Среднее NPV:', np.mean(npv_all3))
print('Среднее IRR:', np.mean(irr_all3))
        

Среднее NPV: 204805.27135887084
Среднее IRR: 0.13129772956073174


## Timing

In [14]:
def wrapper(func, args, kwargs):
    def wrapped():
        return func(args, kwargs)
    return wrapped

wrapped1 = wrapper(npv_mc, 1, 45000)

print('Время:', timeit.timeit(wrapped1, number=1))    


wrapped2 = wrapper(npv_mc_no, 1, 45000)

print('Время:', timeit.timeit(wrapped2, number=1))    



wrapped3 = wrapper(npv_mc_no_no, 1, 45000)

print('Время:', timeit.timeit(wrapped3, number=1))    


Время: 16.052938254000765
Время: 8.992007501001353
Время: 11.486441868000838
